In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, StorageContext, load_index_from_storage
import os
from langchain import HuggingFaceHub
import sys
import logging
import torch

sys.path.append('../')
from apikeys import huggingface_key
from huggingface_hub import InferenceClient
import tqdm as notebook_tqdm

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key

In [4]:
client = InferenceClient("mistralai/Mixtral-8x7B-Instruct-v0.1")

In [5]:
def format_prompt(message, history):
    prompt = "<s>"
    for user_prompt, bot_response in history:
        prompt += f"[INST] {user_prompt} [/INST]"
        prompt += f" {bot_response}</s> "
        prompt += f"[INST] {message} [/INST]"
    return prompt

In [6]:
def format_prompt2(message):
    prompt = "<s>"
    prompt += f"[INST] {message} [/INST]"
    prompt += f" </s> "
        #prompt += f"[INST] {message} [/INST]"
    return prompt

In [7]:
def generate(
    prompt, history, temperature=0.2, max_new_tokens=256, top_p=0.95, repetition_penalty=1.0,
):
    temperature = float(temperature)
    if temperature < 1e-2:
        temperature = 1e-2
    top_p = float(top_p)

    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        seed=42,
    )

    formatted_prompt = format_prompt2(prompt)
    print(f"prompt: {formatted_prompt}")

    stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=True, details=True, return_full_text=False)
    output = ""

    for response in stream:
        output += response.token.text
        yield output
    return output

In [8]:
for line in generate("how sleep should a newborn get?", [("","")]):
    continue

prompt: <s>[INST] how sleep should a newborn get? [/INST] </s> 


In [9]:
print(line)


Newborns generally require a lot of sleep to support their rapid growth and development. On average, newborns (0-3 months old) need about 14-16 hours of sleep per day. However, it's important to note that newborns have not yet established a regular sleep-wake cycle, so they may sleep for short periods of time throughout the day and night.

During the first few weeks of life, newborns may only sleep for 1-2 hours at a time before waking up to feed. As they grow and develop, their sleep patterns will become more predictable, and they may start to sleep for longer stretches at night.

It's also important to note that newborns should be put to sleep on their backs on a firm, flat surface to reduce the risk of sudden infant death syndrome (SIDS). Parents and caregivers should also avoid letting newborns sleep for long periods of time in car seats, strollers, or other devices, as this can increase the risk of suffocation or other injuries.

If you have concerns about your newborn's sleep pa

In [10]:
from llama_index.prompts import PromptTemplate

template = (
    "<s> [INST] We have provided context information below. If information does not provide enough information to ask the question respond with not enough context\n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n [/INST] </s> "
)
qa_template = PromptTemplate(template)

In [11]:
from llama_index.llms import HuggingFaceInferenceAPI
mixtral = HuggingFaceInferenceAPI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1", token=huggingface_key
)

In [12]:
service_context = ServiceContext.from_defaults(
    llm=mixtral, embed_model="local:BAAI/bge-small-en-v1.5"
)

In [13]:
'''
llm = LLMPredictor(llm=HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",verbose=True))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
'''

'\nllm = LLMPredictor(llm=HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",verbose=True))\nservice_context = ServiceContext.from_defaults(\n    llm_predictor=llm, embed_model="local:BAAI/bge-small-en-v1.5"\n)\n'

In [14]:
# load the documents and create the index
documents = SimpleDirectoryReader("../data").load_data()
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [15]:
from llama_index.llms import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the question: {query_str}\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "We have the opportunity to refine the original answer "
            "(only if needed) with some more context below.\n"
            "------------\n"
            "{context_msg}\n"
            "------------\n"
            "Given the new context, refine the original answer to better "
            "answer the question: {query_str}. "
            "If the context isn't useful, output the original answer again.\n"
            "Original Answer: {existing_answer}"
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [16]:

print(
    index.as_query_engine(
        text_qa_template=text_qa_template, refine_template=refine_template, similarity_top_k=6
    ).query("how much sleep should a newborn get?")
)
#query_engine = index.as_query_engine()
#response = query_engine.query("how much should a one month old drink?")
#print(response)

 Based on the provided context, a newborn should ideally get around 16-20 hours of sleep in a 24-hour period. This is divided into several naps throughout the day and night, with the longest stretch of sleep typically occurring at night. It's important to establish healthy sleep habits early on, such as creating a sleep-conducive environment, maintaining a consistent bedtime routine, and ensuring there's enough time between feedings and sleep.


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("how much sleep should a newborn get?")
print(response)

16 -20 hours a day.


In [18]:
eval_questions = [
    "when should you worry about baby split up?",
    "how often should you feed your baby?",
    "how much sleep should your baby get?",
    "how often should a baby poop?",
    "how should a clean my baby?"
]

In [19]:
for question in eval_questions:
    print(
        index.as_query_engine(
            text_qa_template=text_qa_template, refine_template=refine_template, similarity_top_k=6
        ).query(question)
    )
    print("------------------------")


Based on the new context provided, it seems that you are interested in knowing when to seek medical attention for a crying baby. While the original answer I provided earlier is still valid, I will refine it to better fit the context.

You should worry about a baby's crying and consider seeking medical attention if it is persistent (crying for more than 3 hours a day, three or more days a week), accompanied by signs of distress such as abdominal pain, a bright red or purple pin-prickly rash that does not blanch to white when you apply pressure, hive-like rash all over with facial swelling, difficulty breathing, or lips turning blue or purple. These could be signs of serious conditions such as colic or other health issues. However, if your child is gaining weight on schedule and exhibiting repeated, persistent crying, it's important to consult with your pediatrician to rule out other causes for your infant's distress.
------------------------


Refined Answer:

According to the provided